In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
import time
import tqdm
import sys
sys.path.append('../..')
from comparisonHC import HandlerTriplets, OracleTriplets, get_AddS_triplets, get_MulK_triplets, get_tSTE_triplets, ComparisonHC

In [ ]:
#load the datapoints
df = pd.read_csv("../../resources/reduced_test_X.csv",names=['X', 'Y'])

In [ ]:
#load the labels
df_label = pd.read_csv("../../resources/test_labels.csv",names=['Label'])

In [ ]:
df = pd.concat([df,df_label],axis=1)

In [ ]:
#creating a dataset with 200 datapoints from each class
df_reduced = df[df['Label']==0]

In [ ]:
df_reduced = df_reduced.sample(n=200)

In [ ]:
df_reduced.reset_index(inplace=True,drop=True)

In [ ]:
for i in range(1,10):
    df_dummy = df[df['Label']==i]
    df_dummy = df_dummy.sample(n=200)
    df_dummy.reset_index(inplace=True,drop=True)
    df_reduced = pd.concat([df_reduced,df_dummy],axis=0)
    df_reduced.reset_index(inplace=True,drop=True)

In [ ]:
df_label = df_reduced['Label']

In [ ]:
df_reduced = df_reduced.drop('Label',axis=1)

In [ ]:
#normalise the features
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(df_reduced)
df_reduced = scaler.transform(df_reduced)

In [ ]:
#get the similarity matrix
Cos_sim = sklearn.metrics.pairwise.cosine_similarity(df_reduced, dense_output=True)

In [ ]:
#objects in the data
n = Cos_sim.shape[0]
print(n)

In [ ]:
Oracle = OracleTriplets(Cos_sim,n,n_triplets=int(n*n),proportion_noise=0.05)

In [ ]:
adds_similarities = get_AddS_triplets(Oracle,n)

In [ ]:
mulk_similarities = get_MulK_triplets(Oracle,n)

In [ ]:
tste_similarities = get_tSTE_triplets(Oracle,n)

In [ ]:
chc = ComparisonHC(adds_similarities,n)
chc.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc.time_elapsed))
print("Revenue with AddS-3: {}".format(chc.cost_dasgupta(adds_similarities)))

In [ ]:
chc_tste = ComparisonHC(tste_similarities,n)
chc_tste.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc_tste.time_elapsed))
print("Revenue with t-STE: {}".format(chc_tste.cost_dasgupta(adds_similarities)))

In [ ]:
chc_mulk = ComparisonHC(mulk_similarities,n)
chc_mulk.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc_mulk.time_elapsed))
print("Revenue with MulK-3: {}".format(chc_mulk.cost_dasgupta(adds_similarities)))

In [ ]:
chc_cos = ComparisonHC(Cos_sim,n)
chc_cos.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc_cos.time_elapsed))
print("Revenue with Cosine Similarity: {}".format(-chc_cos.cost_dasgupta(adds_similarities)))